# 4장 단어 수준 임베딩
## 4.1 NPLM

Neural Probabilistic Language Model(NPLM)은 자연어 처리 분야에 임베딩 개념을 널리 퍼뜨리는 데 일조한 선구자적 모델로, 딥러닝의 아버지 요슈아 벤지오 연구 팀이 제안한 기법(Bengio et al., 2003)이다.


### 모델 기본 구조
NPLM은 '단어가 어떤 순서로 쓰였는가'에서 설명한 통계 기반의 전통적인 언어 모델의 한계를 극복하는 과정에서 탄생했다. Bengio et al.(2003)은 기존 언어 모델의 단점을 다음과 같이 설명했다.

- 학습 데이터에 존재하지 않는 n-gram이 포함된 문장이 나타날 확률 값을 0으로 부여한다. 물론 **백오프**나 **스무딩**으로 이런 문제를 일부 보완할 수 있지만 완전한 것은 아니다.
- 문장의 **장기 의존성(long-term dependency)**를 포착해내기 어렵다. 다시 말해 n-gram 모델의 n을 5 이상으로 길게 설정할 수 없다. n이 커질수록 그 등장 확률이 0인 단어 시퀀스가 기하급수적으로 늘어난다. 
- 단어/문장 간 유사도를 계산할 수 없다. 

### NPLM의 학습
학습 파라미터의 종류가 많고 그 크기도 큰 편이다. 

## 4.2 Word2Vec
2013년 구글 연구 팀이 발표한 기법으로 가장 널리 쓰이고 있는 단어 임베딩 모델이다. 두 개의 논문으로 나누어 발표했는데, 하나는 Skip-Gram과 CBOW라는 모델이 제안됐고 다른 하나는 이 두 모델을 근간으로 하되 네거티브 샘플링 등 학습 최적화 기법을 제안한 내용이 핵심 골자이다. 

### 모델 기본 구조
- CBOW는 주변에 있는 **문맥 단어**들을 가지고 **타깃 단어** 하나를 맞추는 과정에서 학습된다. 
- Skip-gram 모델은 타깃 단어를 가지고 주변 문맥 단어가 무엇일지 예측하는 과정에서 학습된다. 
- CBOW의 경우 입력, 출력 학습 데이터 쌍이 {문맥 단어 4개, 타깃 단어} 하나인 반면, Skip-gram의 학습 데이터는 {타깃 단어, 타깃 직전 두 번째 단어}, {타깃 단어, 타깃 직전 단어}, {타깃 단어, 타깃 다음 단어}, {타깃 단어, 타깃 다음 두 번째 단어} 이렇게 4개쌍이 된다. Skip-gram이 같은 말뭉치로도 더 많은 학습 데이터를 확보할 수 있어서 임베딩 품질이 CBOW보다 좋은 경향이 있다. 따라서 Skip-gram 모델을 중심으로 Word2Vec 기법을 설명한다. 

In [1]:
# 튜토리얼

#git pull origin master
#bash preprocess.sh dump-tokenized

#cd /notebooks/embedding
#cat data/tokenized/wiki_ko_mecab.txt data/tokenized/ratings_mecab.txt data/tokenized/korquad_mecab.txt > data/tokenized/corpus_mecab.txt 

In [ ]:
# Word2Vec Skip-gram 모델 학습

corpus_fname = '/notebooks/embedding/data/tokenized/corpus_mecab.txt'
model_fname = '/notebooks/embedding/data/word-embbedings/word2vec/word2vec'

from genism.models import Word2Vec

corpus = [sent.strip().split(" ") for sent in open(corpus_fname, 'r').readlines()]
model = Word2Vec(corpus, size=100, workers=4, sg=1)
model.save(model_fname)

In [ ]:
# Word2Vec Skip-gram 모델 학습 스크립트
#cd /notebooks/embedding
#python models/word_utils.py --method train_word2vec \
--input_path data/tokenized/corpus_mecab.txt \
--input_path data/word-embeddings/word2vec/word2vec

In [ ]:
# 코사인 유사도 상위 단어 목록 체크 코드

from models.word_eval import WordEmbeddingEvaluator

model = WordEmbeddingEvaluator("/notebooks/embedding/data/word-embbedings/word2vec/word2vec", method="word2vec", dim=100, tokenizer_name="mecab")
model.most_similar("희망", topn=5)

#### 코사인 유사도 유의점
코사인 유사도 값이 높다도 해서 두 단어가 유의 관계인 것은 아니다. 예를 들어 '덥다'와 '춥다'는 반의 관계이지만, '기온'이라는 속성을 매개로 강한 관련을 맺고 있기 때문에 코사인 유사도 값이 높다. **관련성**이 높은 단어를 출력한다는 의미로 이해하는 것이 좋다. 